In [ ]:
# © Crown Copyright GCHQ
#
# Licensed under the GNU General Public License, version 3 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.gnu.org/licenses/gpl-3.0.en.html
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# This notebook is not compiled into the documentation due to the time taken to run it to get
# a representative analysis. Please run this notebook locally if you wish to see the outputs.

In [ ]:
# sphinx ignore

import sys

sys.path.append("../..")

%config Completer.use_jedi = False

In [ ]:
random_seed = 1_989

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from gpytorch.kernels import MaternKernel, ScaleKernel
from sklearn.preprocessing import StandardScaler

from vanguard.datasets.bike import BikeDataset
from vanguard.models import InducingPointKernelGPModel
from vanguard.uncertainty import GaussianUncertaintyGPController
from vanguard.vanilla import GaussianGPController
from vanguard.warps import SetWarp, warpfunctions

In [ ]:
DATASET = BikeDataset(rng=np.random.default_rng(random_seed))

In [ ]:
plt.hist(DATASET.train_y.numpy(force=True))
plt.xlabel("$y$", fontsize=15)
plt.show()

In [ ]:
scaler = StandardScaler()
scaled_train_x = scaler.fit_transform(DATASET.train_x.numpy(force=True))
scaled_test_x = scaler.transform(DATASET.test_x.numpy(force=True))

In [ ]:
class SparseGaussianGPController(GaussianGPController):
    gp_model_class = InducingPointKernelGPModel

In [ ]:
N_INDUCING_POINTS = 50
num_iters = int(len(scaled_train_x) / 64) * 10

In [ ]:
class ScaledMaternKernel(ScaleKernel):
    """A scaled matern kernel."""

    def __init__(self):
        super().__init__(MaternKernel(nu=1.5, ard_num_dims=scaled_train_x.shape[1]))

In [ ]:
gp = SparseGaussianGPController(
    train_x=scaled_train_x,
    train_y=DATASET.train_y,
    kernel_class=ScaledMaternKernel,
    y_std=DATASET.train_y_std,
    gp_kwargs={"n_inducing_points": N_INDUCING_POINTS},
    optim_kwargs={"lr": 0.01},
    rng=np.random.default_rng(random_seed),
)

gp.fit(n_sgd_iters=num_iters)

In [ ]:
posterior = gp.posterior_over_point(scaled_test_x)
mean, lower, upper = posterior.confidence_interval()

# Convert to numpy arrays for plotting
plt_test_y = DATASET.test_y.numpy(force=True)
mean = mean.numpy(force=True)
lower = lower.numpy(force=True)
upper = upper.numpy(force=True)

print(f"RMSE: {np.sqrt(np.mean((plt_test_y - mean) ** 2))}")
plt.errorbar(plt_test_y, mean, yerr=np.vstack([mean - lower, upper - mean]), marker="o", label="mean", linestyle="")
plt.xlabel("true y values")
plt.ylabel("predicted y values")
plt.legend()
plt.show()

In [ ]:
warp = warpfunctions.AffineWarpFunction() @ warpfunctions.BoxCoxWarpFunction(lambda_=0)


@SetWarp(warp_function=warp, ignore_methods=("fit", "__init__"))
class WarpedGaussianGPController(SparseGaussianGPController):
    pass

In [ ]:
gp = WarpedGaussianGPController(
    train_x=scaled_train_x,
    train_y=DATASET.train_y,
    kernel_class=ScaledMaternKernel,
    y_std=DATASET.train_y_std,
    gp_kwargs={"n_inducing_points": N_INDUCING_POINTS},
    optim_kwargs={"lr": 0.01},
    rng=np.random.default_rng(random_seed),
)

gp.fit(n_sgd_iters=num_iters)

In [ ]:
posterior = gp.posterior_over_point(scaled_test_x)
warp_mean, warp_lower, warp_upper = posterior.confidence_interval()

# Convert to numpy arrays for plotting
warp_mean = warp_mean.numpy(force=True)
warp_lower = warp_lower.numpy(force=True)
warp_upper = warp_upper.numpy(force=True)


print(f"RMSE: {np.sqrt(np.mean((plt_test_y - warp_mean) ** 2))}")
plt.errorbar(
    plt_test_y,
    mean,
    yerr=np.vstack([warp_mean - warp_lower, warp_upper - warp_mean]),
    marker="o",
    label="mean",
    linestyle="",
)
plt.xlabel("true y values")
plt.ylabel("predicted y values")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
small_indices = plt_test_y < 0.5

plt.errorbar(
    plt_test_y[small_indices],
    mean[small_indices],
    yerr=np.vstack([mean - lower, upper - mean])[:, small_indices],
    marker="o",
    label="mean",
    linestyle="",
)
plt.title(f"No warping. RMSE: " f"{np.sqrt(np.mean((plt_test_y[small_indices] - mean[small_indices]) ** 2)):.4}")
plt.xlabel("true y values")
plt.ylabel("predicted y values")

plt.subplot(1, 2, 2)
y_err = np.vstack([warp_mean - warp_lower, warp_upper - warp_mean])[:, small_indices]
plt.errorbar(plt_test_y[small_indices], warp_mean[small_indices], yerr=y_err, marker="o", label="mean", linestyle="")
plt.title(
    f"Affine-log warping. RMSE: {np.sqrt(np.mean((plt_test_y[small_indices] - warp_mean[small_indices]) ** 2)):.4}"
)
plt.xlabel("true y values")

plt.tight_layout()
plt.show()

In [ ]:
warp = warpfunctions.AffineWarpFunction() @ warpfunctions.BoxCoxWarpFunction(lambda_=0)


@SetWarp(warp_function=warp, ignore_all=True)
class WarpedGaussianUncertaintyGPController(GaussianUncertaintyGPController):
    pass

In [ ]:
gp = WarpedGaussianUncertaintyGPController(
    train_x=scaled_train_x[:500],
    train_x_std=0.1,
    train_y=DATASET.train_y[:500],
    kernel_class=ScaledMaternKernel,
    y_std=0.001 * torch.mean(torch.abs(DATASET.train_y)),
    likelihood_kwargs={"learn_additional_noise": True},
    batch_size=None,
    rng=np.random.default_rng(random_seed),
)
gp.fit(n_sgd_iters=num_iters)

In [ ]:
posterior = gp.posterior_over_point(scaled_test_x)
mean, lower, upper = posterior.confidence_interval()

# Convert to numpy arrays for plotting
mean = mean.numpy(force=True)
lower = lower.numpy(force=True)
upper = upper.numpy(force=True)

print(f"RMSE: {np.sqrt(np.mean((plt_test_y - mean) ** 2))}")
plt.errorbar(plt_test_y, mean, yerr=np.vstack([mean - lower, upper - mean]), marker="o", label="mean", linestyle="")
plt.xlabel("true y values")
plt.ylabel("predicted y values")
plt.legend()
plt.show()